In [1]:
import os

file_path = os.path.abspath(".")
execution_path = os.path.abspath(f"../../../../Binaries/Debug")

In [ ]:
import os

os.chdir(file_path)
import sys

target_path = os.path.abspath(f"../../../../Binaries/Debug")
sys.path.append(target_path)
print(f"Added {target_path} to sys.path")
package_path = os.path.abspath(f"../python")
sys.path.append(package_path)
print(f"Added {package_path} to sys.path")

In [3]:
import glints.scratch_grid
import glints.renderer
import glints.test_utils as test_utils
import torch
import numpy as np
import pytest

In [ ]:
import importlib

importlib.reload(glints.scratch_grid)

In [5]:
def linear_to_gamma(image):
    return image ** (1.0 / 2.2)


def gamma_to_linear(image):
    return image**2.2


import matplotlib.pyplot as plt


def save_images(field, resolution, divergence, smoothness):
    for i in range(field.field.shape[2]):
        test_utils.save_image(
            1000 * divergence[:, :, i], resolution, f"divergence_{i}.exr"
        )
        test_utils.save_image(
            100 * smoothness[:, :, i], resolution, f"smoothness_{i}.exr"
        )

        density = torch.norm(field.field[:, :, i], dim=2)
        directions = field.field[:, :, i] / density.unsqueeze(2)
        directions = torch.cat(
            [directions, torch.zeros_like(directions[:, :, :1])], dim=2
        )

        test_utils.save_image(directions, resolution, f"directions_{i}.exr")
        test_utils.save_image(density, resolution, f"density_{i}.exr")
        test_utils.save_image(field.field[:, :, i, :1], resolution, f"field_{i}.exr")

In [ ]:
os.chdir(execution_path)
r = glints.renderer.Renderer()

vertices, indices = glints.renderer.plane_board_scene_vertices_and_indices()
camera_position_np = np.array([0.0, 0.0, 7], dtype=np.float32)
r.set_look_at(camera_position_np, np.array([0.0, 0.0, 0.0], dtype=np.float32), np.array([-1.0, 0.0, 0.0], dtype=np.float32))
#r.set_camera_position(camera_position_np)
fov_in_degrees = 35
resolution = [768 * 2, 512 * 2]
r.set_perspective(
    np.pi * fov_in_degrees / 180.0, resolution[0] / resolution[1], 0.1, 1000.0
)
r.set_mesh(vertices, indices)
r.set_light_position(torch.tensor([4.0, 1.0, 4.5], device="cuda"))

r.set_width(torch.tensor([0.001], device="cuda"))

field = glints.scratch_grid.ScratchField(512, 1)
image, sampled_mask = glints.scratch_grid.render_scratch_field(r, resolution, field)
test_utils.save_image(image, resolution, "scratch_field_initial.exr")
target_image = r.prepare_target("texture.png", resolution)
#target_image = test_utils.read_image("render_010.exr")

loss_fn = torch.nn.MSELoss()
regularization_loss_fn = torch.nn.HuberLoss()
regularizer = torch.optim.Adam([field.field], lr=0.005)
optimizer = torch.optim.Adam([field.field], lr=0.02)

In [ ]:
losses = glints.scratch_grid.optimize_field(
    field,
    r,
    resolution,
    gamma_to_linear(target_image),
    loss_fn,
    regularization_loss_fn,
    regularizer,
    optimizer,
    epochs=500,
)

plt.plot(losses)
plt.show()

In [15]:
test_utils.save_image(gamma_to_linear(target_image), resolution, "target_image.exr")

In [ ]:
divergence, smoothness = field.calc_divergence_smoothness()
save_images(field, resolution, divergence, smoothness)

image, sampled_mask = glints.scratch_grid.render_scratch_field(r, resolution, field)
test_utils.save_image(image, resolution, "scratch_field.exr")

directions = torch.rot90(field.field[:, :, 0, :2])


test_utils.plot_arrows(directions, "directions", spacing=6, filename="directions.pdf")

In [17]:
field.field.requires_grad = False
field.field = field.field.detach()
torch.cuda.empty_cache()

In [18]:
# line_counts = []
# for density_held in np.linspace(2.0, 3.0, 9):
#     lines = field.discretize_to_lines(density_held, 0.4)
#     line_counts.append(lines.shape[0])


# plt.plot(np.linspace(2.0, 3.0, 9), line_counts)
# plt.xlabel("Density held")
# plt.ylabel("Line count")
# plt.savefig("line_count.pdf")
# plt.show()

In [ ]:
lines = field.discretize_to_lines(0.6, 0.5)
print(lines.shape)
print(lines)

In [20]:
# os.chdir(execution_path)
# import torch
# lines = torch.load("lines.pt")

# r = glints.renderer.Renderer()

# vertices, indices = glints.renderer.plane_board_scene_vertices_and_indices()
# camera_position_np = np.array([4.0, 0.0, 3.5], dtype=np.float32)
# r.set_camera_position(camera_position_np)
# fov_in_degrees = 35
# resolution = [768 * 2, 512 * 2]
# r.set_perspective(
#     np.pi * fov_in_degrees / 180.0, resolution[0] / resolution[1], 0.1, 1000.0
# )
# r.set_mesh(vertices, indices)
# r.set_light_position(torch.tensor([0.0, 4.0, 4.5], device="cuda"))

# r.set_width(torch.tensor([0.001], device="cuda"))

In [21]:
#torch.cuda.empty_cache()
r.set_type("bspline")

image, _ = r.render(resolution, lines)
test_utils.save_image(image, resolution, "lines.exr")